# Deep ViT Features - Point Correspondences
Given a pair of images, find $k$ semantic correspondences between them.

In [1]:
#@title Installations and mounting
import sys
sys.path.append('dino-vit-features')
%load_ext autoreload
%autoreload 2

## Change Runtime Type
To get a GPU in Google Colab, go to the top menu: Runtime ➔ Change runtime type and select GPU as Hardware accelerator.

In [3]:
#@title Configuration:
#@markdown Choose image paths:

image_path1 = '/scratch.global/chen7924/cropped_NGMDB_GF/10069_1.tif' #@param
image_path2 = '/scratch.global/chen7924/topomaps_0101/topomaps_1130/CA_Piru_294178_1952_24000_geo.tif' #@param

#@markdown Choose number of points to output:
num_pairs = 10 #@param
#@markdown Choose loading size:
load_size = 224 #@param
#@markdown Choose layer of descriptor:
layer = 9 #@param
#@markdown Choose facet of descriptor:
facet = 'key' #@param
#@markdown Choose if to use a binned descriptor:
bin=True #@param
#@markdown Choose fg / bg threshold:
thresh=0.05 #@param
#@markdown Choose model type:
model_type='dino_vits8' #@param
#@markdown Choose stride:
stride=4 #@param

In [6]:
import matplotlib.pyplot as plt
import torch
from correspondences import find_sims, draw_correspondences

with torch.no_grad():
    sims = find_sims(image_path1, image_path2, num_pairs, load_size, layer, facet, bin, thresh, model_type, stride)
    print(sims)
    # points1, points2, image1_pil, image2_pil = find_correspondences(image_path1, image_path2, num_pairs, load_size, layer,
    #                                                                facet, bin, thresh, model_type, stride)
# fig_1, ax1 = plt.subplots()
# ax1.axis('off')
# ax1.imshow(image1_pil)
# fig_2, ax2 = plt.subplots()
# ax2.axis('off')
# ax2.imshow(image2_pil)


# fig1, fig2 = draw_correspondences(points1, points2, image1_pil, image2_pil)
# plt.show()

Using cache found in /home/yaoyi/chen7924/.cache/torch/hub/facebookresearch_dino_main


torch.Size([1, 3, 281, 224])
torch.Size([1, 3, 273, 224])
1.5810898952186108


In [1]:
from PIL import Image
import os
import numpy as np
import pickle
import sys
import argparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from correspondences import find_sims
import multiprocessing
from tqdm import tqdm
from functools import partial

def calc_geo_topo_correlation(gt_pair, geo_root, topo_root):# Paths to your images
    geo_name, topos = gt_pair
    geo_root = geo_root
    topo_root = topo_root
    geomap_path = os.path.join(geo_root, geo_name +'.tif')
    print(geomap_path)
    
    
    #For each geo-topo pair calculate, bbp sum similarity
    sim_scores = []
    for topo in topos:
        topomap_path = os.path.join(topo_root, topo + '.tif')
        print(topomap_path)
        sim = find_sims(geomap_path, topomap_path)
        sim_scores.append(sim)
    
    combined = list(zip(sim_scores, topos))
    sorted_combined_list = sorted(combined_list, key=lambda x: x[0], reverse=True)

    # Extract the sorted lists of integers and strings
    sorted_integers, sorted_strings = zip(*sorted_combined_list)
    
    print(sorted_strings[0])
    print(gt_true_dict[geo_name])
    sys.exit()
    return (geo_name, sorted_strings)


/home/yaoyi/chen7924/anaconda3/envs/gp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
geo_root = '/scratch.global/chen7924/cropped_NGMDB_GF'
topo_root = '/scratch.global/chen7924/topomaps_0101/topomaps_1130'
out_list = '/scratch.global/chen7924/semcorr_results/dino_vit_pairs.pkl'

#load in dict with main pairs
# with open('/home/yaoyi/chen7924/critical-maas/models/vit-topo/data/mn_geo-topo_pairs_90OR90.pickle', 'rb') as handle:
#     geo_topo_pairs = pickle.load(handle)

#Load in dict with incorrect rank 20 pairs
with open('/home/yaoyi/chen7924/critical-maas/Data/semantic_correspondence/sem_corr_1-1_top20_f.pkl', 'rb') as handle:
    rank20_dict_corr = pickle.load(handle)

rank20_dict = {}
for geo_name in rank20_dict_corr.keys():
    geomap_path = os.path.join(geo_root, geo_name +'.tif')
    if os.path.exists(geomap_path):
        rank20_dict[geo_name] = rank20_dict_corr[geo_name]
# del rank20_dict_corr
# print(len(list(rank20_dict.keys())))
# # Function to save strings to a pickle file
# def save_strings(strings):
#     with open(out_list, "wb") as file:
#         pickle.dump(strings, file)   

# results_list = []
# strings_list = []
# loop_counter = 0
# for key, value in rank20_dict.items():
#     pairs = calc_geo_topo_correlation((key, value), geo_root, topo_root)
#     strings_list.append(pairs)
#     loop_counter += 1
#     if loop_counter % 100 == 0:
#         print(f"Accuracy after {loop_counter} threads: {sum(results_list)/len(results_list)}")
#         # Save the strings every 100 threads
#         save_strings(sqtrings_list)

# save_strings(strings_list)
# print(f"Accuracy after {loop_counter} threads: {sum(results_list)/len(results_list)}")